<a href="https://colab.research.google.com/github/arpdm/predictive-maintenance-platform/blob/main/PdM_NASA_ENGINES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab data file preparation
from google.colab import drive

drive.mount("/content/drive")

!cp drive/MyDrive/Predictive_Maintenence_Fault_Detection/predictive-maintenance-platform/data_processor.py .
!cp drive/MyDrive/Predictive_Maintenence_Fault_Detection/predictive-maintenance-platform/visualizer_analyzer.py .


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tabnanny import verbose
from data_processor import DataProcessor
from visualizer_analyzer import DataAV

import tensorflow as tf

# Load all datasets
DS_001 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS01-005.h5"
DS_002 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS02-006.h5"
DS_003 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS03-012.h5"
DS_004 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS04.h5"
DS_005 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS05.h5"
DS_006 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS06.h5"
DS_007 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS07.h5"
DS_008 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS08a-009.h5"
DS_009 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS08c-008.h5"
DS_010 = "/content/drive/MyDrive/Predictive_Maintenence_Fault_Detection/data_set/N-CMAPSS_DS08d-010.h5"

In [ ]:
# Load data set and prepare data frames
pros = DataProcessor()
pros.load_hdf5_to_numpy_arr(DS_004)

In [ ]:
# Create a visualizer class. This is useful for understanding the data we work with
# it will determine feature extraction and model architecture to some extend
vs_an = DataAV(pros.df_aux,pros.df_x_s,pros.x_s_var_names,pros.df_w,pros.df_t,pros.t_var_names,pros.w_var_names,pros.df_ts)

In [ ]:
pros.x_s_dev

In [ ]:
# Generate all visualization graphs and plots necessary for understanding data.
vs_an.get_engine_units_in_dataset()
vs_an.plot_flight_classes()
vs_an.show_engine_health_parameter_stats()
vs_an.generate_engine_health_parameter_graphs()
vs_an.generate_hpt_eff_over_cycles_all_engines()
vs_an.generate_sensor_readings_graphs_single_unit(1)
vs_an.generate_sensor_readings_graphs_single_unit_single_cycle(1, 2)
vs_an.plot_health_states_for_all_engines()
vs_an.generate_flight_profle_single_unit_single_cycle(1, 2)
vs_an.generate_flight_envelope()
vs_an.generate_kde_estimations_of_flight_profile()

In [5]:
"""
Genearate the tf dataset with proper dimensionality and shape given input parameters.
This dataset will split into training and validation subsets.
"""

window = 48
horizon = 1
train_split = 6377452
batch_size = 256
buffer_size = 150

x_train, y_train = pros.custom_ts_multi_data_prep(pros.w, pros.y_rul, 0, train_split, window, horizon)
x_vali, y_vali = pros.custom_ts_multi_data_prep(pros.w, pros.y_rul, train_split, None, window, horizon)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat()


In [ ]:
"""
Build the DNN model with its layers that will be used for RUL preditcions.
"""

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(200, return_sequences=True), input_shape=x_train.shape[-2:]
        ),
        tf.keras.layers.Dense(20, activation="tanh"),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
        tf.keras.layers.Dense(20, activation="tanh"),
        tf.keras.layers.Dense(20, activation="tanh"),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1),
    ]
)

model.compile(optimizer="sgd", loss="mae")
model.summary()

In [ ]:
# Model - 2 That uses Convolution, LSTM, and Dense

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape= x_train.shape[-2:]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="sigmoid"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

model.compile(optimizer="adam", loss="mae", metrics=['accuracy'])
model.summary()

In [ ]:
"""
Train the generate model based on provided epochs and steps.
This function will also automatically generate the plot for loss history.
"""

epochs = 60
steps = 100
validation_steps = 50
verbose = 1

tf.keras.backend.clear_session()

history = model.fit(
    x=train_data, epochs=epochs, steps_per_epoch=steps, validation_data=val_data, validation_steps=validation_steps, verbose=verbose
)

vs_an.plot_training_results_history(history)

In [ ]:
y_pred = model.predict(x_vali)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,9))
plt.plot( list(y_vali.reshape(-1)))
plt.plot( list(y_pred.reshape(-1)))
plt.title("Actual vs Predicted")
plt.ylabel("Traffic volume")
plt.legend(('Actual','predicted'))
plt.show()

In [11]:
test_mae_loss = np.mean(np.abs(y_pred.reshape(1,-1) - y_vali.reshape(1,-1)), axis=1)

In [ ]:
test_mae_loss